In [ ]:
    est = str(estimator)
    if est.find('ComplementNB')>=0 or est.find('ComplementNB')>=0:
        X_train_fs = fs_selector.transform(X_train)
        # when Classifiers don't have 'get_support()' - then call this
        top_feat_fs = get_selected_columns(X_train, X_train_fs, labels, show_compare=True)
    else:
        # non GaussianNB
        X_train_fs = fs_selector.transform(X_train)
        top_feat_fs = get_selected_columns(X_train, X_train_fs, labels, show_compare=True)


In [ ]:
    # combine final "test" Id column with prediction probabilities column (cover to dataframe first) 
    frames = [X_final.iloc[:,0], pd.DataFrame(y_final_preds[:,1])]
    result = pd.concat(frames, axis=1) 
    result.columns = ['Id','tmp']
    result['TARGET_5Yrs'] = [round(num, 2) for num in result['tmp']]
    result.drop(['tmp'], axis=1, inplace=True)

    #--------------------------------------------------------------------------
    # Save the final predictions for submission to Kaggle
    result.to_csv('../data/processed/group1_r{d}_{c}.csv'.format(d=test_no, c=classifier_name), index=False)
    #--------------------------------------------------------------------------


In [ ]:

def save_kaggle_output(classifier, X_train, Y_train, X_final)

    t_start = time.process_time()
    # fit model
    classifier.fit(X_train, Y_train)
    t_end = time.process_time()       
    t_diff = t_end - t_start
    
    
    #save the final "test" prediction probabilities for Kaggle
    y_final_preds = classifier.predict_proba(X_final)

    # combine final "test" Id column with prediction probabilities column (cover to dataframe first) 
    frames = [X_final.iloc[:,0], pd.DataFrame(y_final_preds[:,1])]
    result = pd.concat(frames, axis=1) 
    
    print('result',result)
    
    result.columns = ['Id','tmp']
    result['TARGET_5Yrs'] = [round(num, 2) for num in result['tmp']]
    result.drop(['tmp'], axis=1, inplace=True)

    #--------------------------------------------------------------------------
    # Save the final predictions for submission to Kaggle
    result.to_csv('../data/processed/group1_r{d}_{c}.csv'.format(d=test_no, c=classifier_name), index=False)
    #--------------------------------------------------------------------------
    # prepare data same as for modelling - labels = all features
    #df_clean_mod, df_clean_mod_test = select_features(df_fix_train, df_fix_test, labels)
    #X_train, y_train, X_val, y_val, X_test, y_test = Prep_Model_Data(df_clean_mod, target, scaler=scaler, resample='', random_state=random_state)

    # Run SelectModel Feature Selection for the each Classifier - as they return different number of features
    #top_feat = select_features_auto(X_train, y_train, feature_selector='SFM', no_select_feat=2, estimator=classifier, max_features=i, step=1, cv=5, direction='forward')
    


In [ ]:
# https://towardsdatascience.com/5-feature-selection-method-from-scikit-learn-you-should-know-ed4d116e4172
from sklearn.feature_selection import SequentialFeatureSelector

t_start = time.process_time()

# Selecting the Best important features according to Logistic Regression
sfs_selector = SequentialFeatureSelector(estimator=LogisticRegression(), n_features_to_select=3, cv=5, direction ='backward')
sfs_selector.fit(df_fix_train, target)

print(sfs_selector)

top_feat_sfs = df_fix_train.columns[sfs_selector.get_support()]
print(top_feat_sfs)
print('Time taken:', (time.process_time()-t_start))

In [ ]:
# 4. Feature Selection via SelectFromModel
# https://towardsdatascience.com/5-feature-selection-method-from-scikit-learn-you-should-know-ed4d116e4172
from sklearn.feature_selection import SelectFromModel

t_start = time.process_time()

# Selecting the Best important features according to Logistic Regression using SelectFromModel
sfm_selector = SelectFromModel(estimator=LogisticRegression(), max_features=3)
sfm_selector.fit(df_fix_train, target)

print(sfm_selector)

top_feat_sfm = df_fix_train.columns[sfm_selector.get_support()]
print(top_feat_sfm)
print('Time taken:', (time.process_time()-t_start))

In [ ]:
# Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

# Define the search space for xgboost hyperparameters
space = {
    'max_depth' : hp.choice('max_depth', range(5, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.05),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.05)
}

# Define a function called objective
def objective(space):
    from sklearn.model_selection import cross_val_score
    
    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree'],
        random_state=random_state
    )
    # X_train_res with selected features only
    acc = cross_val_score(xgboost, X_train_res, y_train_res, cv=10, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }


In [ ]:
# Launch Hyperopt search and save the result in a variable called `best`
best = fmin(
    fn=objective,   
    space=space, 
    algo=tpe.suggest,       
    max_evals=10
)

# Print the best set of hyperparameters
print("\nBest: ", best)

In [ ]:
def Train_LogisticRegression(clf_lr, X_train, y_train, X_val, y_val, X_test, y_test):
    # Fit the model with the prepared data
    clf_lr.fit(X_train, y_train)
    print(clf_lr.fit(X_train, y_train),'\n')

    # Import dump from joblib and save the fitted model into the folder models as a file called xgboost_default
    from joblib import dump 
    dump(clf_lr,  '../models/clf_lr_default.joblib')

    # Save the predictions from this model for the training and validation sets into 2 variables called y_train_preds and y_val_preds
    y_train_preds = clf_lr.predict(X_train)
    y_val_preds = clf_lr.predict(X_val)
    y_test_preds = clf_lr.predict(X_test)
    
    # prediction probabilities for AUC
    y_train_pred_prob = clf_lr.predict_proba(X_train)
    y_val_pred_prob = clf_lr.predict_proba(X_val)
    y_test_pred_prob = clf_lr.predict_proba(X_test)

    print_class_perf(y_preds=y_train_preds, y_actuals=y_train, y_pred_prob=y_train_pred_prob[:,1], set_name='Training', average='weighted')
    print_class_perf(y_preds=y_val_preds, y_actuals=y_val, y_pred_prob=y_val_pred_prob[:,1], set_name='Validation', average='weighted')    
    print_class_perf(y_preds=y_test_preds, y_actuals=y_test, y_pred_prob=y_test_pred_prob[:,1], set_name='Test', average='weighted')
    
    return clf_lr, clf_lr

In [ ]:
# Instantiate a LogisticRegression with best set of hyperparameters
lr_2 = LogisticRegression(
    C = best['C'],
    warm_start = warm_start[best['warm_start']],
    fit_intercept = fit_intercept[best['fit_intercept']],
    tol = best['tol'],
    solver = solver_lr[best['solver']],
    max_iter = best['max_iter'],
    intercept_scaling = intercept_scaling[best['intercept_scaling']],
    random_state=random_state
)

In [ ]:
# Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
solver_lr = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
warm_start = [True, False]
fit_intercept = [True, False]
intercept_scaling = [0, 1]
multi_class = ['auto']
class_weight = ['balanced']

# Define the search space for xgboost hyperparameters
space_lr = {
    'C' : hp.uniform('C', 0.05, 3),
    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'solver' : hp.choice('solver', ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']),
    'max_iter' : hp.choice('max_iter', range(100,1000)),
    'intercept_scaling': hp.choice('intercept_scaling', [0, 1])
}

# Define a function called objective
def objective_lr(space_lr):
    from sklearn.model_selection import cross_val_score
    
    clf_lr = LogisticRegression(
        C = space_lr['C'],
        warm_start = space_lr['warm_start'],
        fit_intercept = space_lr['fit_intercept'],
        tol = space_lr['tol'],
        solver = space_lr['solver'],
        max_iter = space_lr['max_iter'],
        intercept_scaling = space_lr['intercept_scaling'],
        random_state=random_state
    )
    # X_train_res with selected features only
    acc = cross_val_score(clf_lr, X_train_res, y_train_res, cv=10, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

In [ ]:
# Launch Hyperopt search and save the result in a variable called `best`
best = fmin(
    fn=objective_lr,   
    space=space_lr, 
    algo=tpe.suggest,       
    max_evals=10
)

# Print the best set of hyperparameters
print("\nBest: ", best)